In [25]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from flask import Flask, render_template, request


In [26]:
app = Flask(__name__)

In [27]:
def get_model():
    global model
    model = load_model('model-017.model')

In [28]:
def load_image(img_path):

    img = image.load_img(img_path, target_size=(255, 255))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    return img_tensor

In [29]:
def prediction(img_path):
    new_image = load_image(img_path)
    
    pred = model.predict(new_image)
    
    labels_dict = {  
        0: 'Tomato_Early_blight',
        1: 'Tomato_Late_blight',
        2: 'Tomato_healthy',
        
        }

    label = np.argmax(pred, axis=1)[0]
    
    return labels_dict[label]

In [30]:
get_model()

In [31]:
@app.route("/", methods=['GET', 'POST'])
def home():

    return render_template('home.html')

@app.route("/predict", methods = ['GET','POST'])
def predict():
    
    if request.method == 'POST':
        
        file = request.files['file']
        filename = file.filename
        file_path = os.path.join(r'D:/my first project/static', filename)                       
        file.save(file_path)
        print(filename)
        product = prediction(file_path)
        print(product)
        
    return render_template('predict.html', product = product, user_image = file_path)    

In [32]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [25/Jun/2022 17:49:44] "POST /predict HTTP/1.1" 200 -


0ae44a6c-1213-4312-a11b-c7c5d4e585d0___RS_Erly.B 9442.JPG
Tomato_Early_blight


127.0.0.1 - - [25/Jun/2022 17:49:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2022 17:49:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2022 17:49:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2022 17:49:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2022 17:49:48] "POST /predict HTTP/1.1" 200 -


0a2726e0-3358-4a46-b6dc-563a5a9f2bdf___RS_Erly.B 7860.JPG
Tomato_Early_blight
